In [3]:
import pandas as pd
import multiprocessing as mp
from gitter.scraper import GitterScraper
from gitter.processing.parallel import parallelize

from gitter.preprocessing.messages import apply_messages_pre_processing, process_messages
from gitter.processing.sentence import apply_sentimental_analysis

In [8]:
try:
    scraper = GitterScraper(
        "62fe76c279230fbd70415c924fef5d1b26f1aec7", "555f74e315522ed4b3e0ce42"
    )
    messages = scraper.get_messages(20, 20)

except ValueError:
    print(f'Error with gitter API')

# Create Dataframe
remove_columns = ["text", "status", "v", "editedAt",
                    "threadMessageCount", "readBy", "unread"]

messages_df = pd.DataFrame(process_messages(messages)).drop(
    columns=remove_columns, errors="ignore")

messages_df.rename(columns={'html': 'sentence'}, inplace=True)
if __name__ == '__main__':
    print('Init apply_messages_pre_processing: ')
    messages_df = parallelize(
        messages_df, apply_messages_pre_processing)


    messages_df.head(5)

Init apply_messages_pre_processing: 
function parallelize executed in 3.3471 seconds


In [37]:
messages_df = pd.read_csv('./data/dataset.csv')
messages_df.set_index('sent', inplace=True)
messages_df.head(5)

,id,sentence
sent,,
2022-04-28 17:50:36.504000+00:00,626ad3ec41f4560c6f7670bd,bow you might want to move to discord for futu...
2022-04-28 17:50:40.964000+00:00,626ad3f0949ae94006807120,this channel is pretty much dead
2022-04-28 17:52:04.718000+00:00,626ad444fa846847c93e1d4a,also: don't use as a type. it has really unin...
2022-04-29 10:53:48.170000+00:00,626bc3bc61f5ca3feeda20d5,is there a convention for variable / functions...
2022-04-29 13:29:19.881000+00:00,626be82fcd938f6ea22273be,what kind of convention


In [38]:
messages_df.groupby(pd.Grouper(freq='D'))

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'